In [1]:
import numpy as np
import pandas as pd
import os
# cuda error 표시 안될 때 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [6]:
import torch
import sklearn
import torchvision
from tqdm import tqdm
from glob import glob
from PIL import Image

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# set seed
torch.manual_seed(1)
if device == "cuda":
    torch.cuda.manual_seed_all(1)

In [4]:
# csv load
import os
data_path = "./2021-ai-w11-p1/"
labelname_csv_path = os.path.join(data_path,"Label2Names.csv")
labelname = pd.read_csv(labelname_csv_path,header=None)
print(labelname)
train_dir_path = os.path.join(data_path,"train_csv_v2")
test_dir_path = os.path.join(data_path,"test_csv_v2")
submission = pd.read_csv(os.path.join(data_path,"submission.csv"))
print(submission)

       0              1
0      1          Faces
1      2     Faces_easy
2      3       Leopards
3      4     Motorbikes
4      5      accordion
..   ...            ...
96    97     wheelchair
97    98       wild_cat
98    99  windsor_chair
99   100         wrench
100  101       yin_yang

[101 rows x 2 columns]
                 Id  Category
0     img_00000.csv        42
1     img_00001.csv        42
2     img_00002.csv        42
3     img_00003.csv        42
4     img_00004.csv        42
...             ...       ...
1707  img_01707.csv        42
1708  img_01708.csv        42
1709  img_01709.csv        42
1710  img_01710.csv        42
1711  img_01711.csv        42

[1712 rows x 2 columns]


In [121]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,split,dir_path,transform,labelname):
        self.split = split
        self.transform = transform
        if self.split == "train":
            self.data_path = glob(os.path.join(dir_path,"*","*"))
            self.data_path = [x.replace("\\","/") for x in self.data_path if "BACKGROUND_Google" not in x]
            self.label = [x.split("/")[-2] for x in self.data_path]
            self.label = [labelname.index[labelname[1] == x].tolist()[0] + 1 for x in self.label]
        else:
            self.data_path = glob(os.path.join(dir_path,"*"))
            self.data_path = sorted(self.data_path)
        self.len = len(self.data_path)
    def __len__(self):
        return self.len
    def __getitem__(self,index):
        img = pd.read_csv(self.data_path[index]).to_numpy()
        img = img.reshape(256,256,3)
        img = img.transpose((2,0,1)).astype(np.int8)
        img = torch.Tensor(img)
        if self.split == "train":
            return img, torch.LongTensor([self.label[index]])
        else:
            return img

In [95]:
from torchvision.transforms import transforms
transform = transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor(), 
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])                            

In [96]:
train_dataset = MyDataset("train",train_dir_path,transform,labelname)
x,y = train_dataset.__getitem__(1)
print(x.shape, y.shape, y)

torch.Size([3, 256, 256]) torch.Size([1]) tensor([5])


In [97]:
# model load
model = torchvision.models.vgg19(pretrained=True)
for param in model.parameters():
    param.requires_grad = False    
model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=101, bias=True)
# init xavier
# torch.nn.init.xavier_normal_(model.classifier[0].weight)
# torch.nn.init.xavier_normal_(model.classifier[3].weight)
torch.nn.init.xavier_normal_(model.classifier[6].weight)
model = model.to(device)
# print(model)

In [98]:
# setting param, optim, cost function,dataloader
from torch.utils.data import DataLoader

lr = 1e-3
batch_size = 16
optim = torch.optim.Adam(model.parameters(), lr=lr)
train_data_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
epochs = 100
loss = torch.nn.CrossEntropyLoss()

In [101]:
def train(model, data_loader):
    model.train()
    sum_cost = 0.0
    sum_correct = 0
    for i,data in enumerate(data_loader):
        data, target = data[0].to(device),data[1].to(device).view(-1)
        optim.zero_grad()
        output = model(data)
        cost = loss(output,target - 1)
        predict = torch.argmax(output,dim=1)
        correct = (predict == target - 1).sum().item()
        sum_cost += cost.item()
        sum_correct += correct
        cost.backward()
        optim.step()
    return sum_cost/len(data_loader.dataset),sum_correct/len(data_loader.dataset)

In [102]:
#train 
import time
cur_time = time.time()
for epoch in range(epochs):
    cost,acc = train(model,train_data_loader)
    print(epoch,cost,acc * 100)
print("endtime =",time.time() - cur_time)

0 2.691008452613755 13.135313531353136
1 1.783296510803424 30.825082508250823
2 1.5056356272681712 39.30693069306931
3 1.2614986295353854 44.554455445544555
4 1.1702696587779735 49.43894389438944
5 1.014270533586886 54.65346534653466
6 0.9437547088062802 57.95379537953795
7 0.9107904307519642 58.97689768976898
8 0.8517425039813857 62.277227722772274
9 0.8132147947836532 62.90429042904291
10 0.8078668486167102 64.12541254125412
11 0.7795673911721006 64.91749174917491
12 0.7556818768529608 66.8976897689769
13 0.7297501772554535 67.98679867986799
14 0.7119215277653717 67.49174917491749
15 0.713341240835662 69.10891089108911
16 0.6604241651719552 70.23102310231023
17 0.670043515155769 71.25412541254126
18 0.640895585976418 72.60726072607261
19 0.6552522220873006 73.53135313531352
20 0.5755583869528468 74.52145214521451
21 0.6153341933201788 73.96039603960396
22 0.6223283769807143 74.38943894389439
23 0.6326249817619869 74.22442244224422
24 0.5861056920122539 75.47854785478549
25 0.62373145

In [105]:
def validate(model, data_loader):
    model.eval()
    sum_correct = 0
    for i,data in enumerate(data_loader):
        output = model(data[0].to(device))
        target = data[1].to(device).view(-1)
        predict = torch.argmax(output,dim=1)
        correct = (predict == target - 1).sum().item()
        sum_correct += correct
    return sum_correct/len(data_loader.dataset)

In [106]:
with torch.no_grad():
    print(validate(model,train_data_loader) * 100)

99.96699669966996


In [122]:
# load test_x
test_dataset = MyDataset("test",test_dir_path,transform,labelname)
test_data_loader = DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=False)

In [137]:
x = test_dataset.__getitem__(1)
print(x.shape)

torch.Size([3, 256, 256])


In [138]:
def predict(model, data_loader):
    model.eval()
    predict_array = []
    for data in data_loader:
        data = data.to(device)
        output = model(data)
        predict = torch.argmax(output,dim=1) + 1
        predict_array += np.array(predict.cpu().detach()).tolist()
    return predict_array

In [139]:
with torch.no_grad():
    predict = predict(model,test_data_loader)
    submission['Category'] = predict
    print(submission)

                 Id  Category
0     img_00000.csv        42
1     img_00001.csv        42
2     img_00002.csv        43
3     img_00003.csv        52
4     img_00004.csv        43
...             ...       ...
1707  img_01707.csv        97
1708  img_01708.csv        11
1709  img_01709.csv        98
1710  img_01710.csv        98
1711  img_01711.csv        82

[1712 rows x 2 columns]


In [140]:
submission.to_csv("submission2.csv",index=False)